Combining the fusion.py code with the ComPat stochastic inducer, III
=======================================================
Here we will not be running the fusion code to steady state, but for dt seconds and then updating the chi.

In [ ]:
%matplotlib inline
#notebook
import numpy as np
import fusion
import ComPat
import scipy.constants  
eV = scipy.constants.eV
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

import scipy.constants
from fipy import Variable, FaceVariable, CellVariable, TransientTerm, DiffusionTerm, Viewer, meshes

**fusion.solve_Te** solves

$$\frac{3}{2}\frac{\partial}{\partial t}\left(n(\rho,t) T(\rho,t)\right) =
    \nabla_\rho \left[ n(\rho,t) \chi(\rho,t) \nabla_\rho
    (T(\rho,t))\right] + S(\rho, t)$$

with a boundary condition given by $Te_{bc}$ and an initial
uniform temperatore of 1000 eV; the quantities are

- $n(\rho,t)$ characterizes the plasma density

- $\chi(\rho,t)$ characterizes the thermal conductivity

- $S(\rho,t)$ characterizes the source

The geometry of the simulation is characterised by the minor radius
$a_0$, major radius $R_0$ and elongation $E_0$
(while the geometry is solved in the cylindrical approximation, the
actual radius used, $a$, is adjusted on the basis of $a_0$
and $E_0$).

These are the standard (default) settings for the fusion code except for setting dt=0.001 and plotting off.

In [ ]:
Qe_tot=2e6        # heating power [W]
H0=0              # position of Gaussian [-]
Hw=0.1            # width of Gaussian [-]
Te_bc=100         # outer edge Te boundary condition [eV]
chi=1             # thermal diffusivity [m^2 s^{-1}]
a0=1              # minor radius [m]
R0=3              # major radius [m]
E0=1.5            # ellipticity
b_pos=0.98        # position of density pedestal [-]
b_height=6e19     # height of density pedestal [m^-3]
b_sol=2e19        # sol value for density pedestal [m^-3]
b_width=0.01      # width of density pedestal [-]
b_slope=0.01      # slope of density pedestal [?]
nr=100            # number of radial grid points
dt=0.01           # time-step [s]
plots=False       # enable FiPy plots

Set up the fusion model.  We will normalise some quantities to the steady state value so we first calculate that.

In [ ]:
Te_ss, ne_ss, rho_ss, rho_norm_ss, Qe_ss = fusion.solve_Te(Qe_tot=Qe_tot, H0=H0, Hw=Hw, Te_bc=Te_bc, chi=chi, 
                                                           a0=a0, R0=R0, E0=E0, 
                                                           b_pos=b_pos, b_height=b_height, b_sol=b_sol, 
                                                           b_width=b_width, b_slope=b_slope, 
                                                           nr=nr, dt=100, plots=False)

In [ ]:
Gd_ss = -np.gradient(Te_ss, rho_ss) * ne_ss * eV
 
Gi_ss = np.cumsum(Qe_ss *  np.diff(rho_ss).mean() * rho_ss) / (rho_ss + np.diff(rho_ss).mean()/2)
Gi_ss = np.append([Gi_ss[0]/2],(Gi_ss[1:]+Gi_ss[:-1])/2)

print(Gi_ss[Gi_ss.shape[0]//2] / Gd_ss[Gd_ss.shape[0]//2])

This is basically copied from fusion.py

In [ ]:
a = a0*np.sqrt(E0)
V = 2*np.pi * 2*np.pi*R0
mesh = meshes.CylindricalGrid1D(nr=nr, Lr=a)
Te = CellVariable(name="Te", mesh=mesh, value=1e3)
ne = CellVariable(name="ne", mesh=mesh, value=fusion.F_ped(mesh.cellCenters.value[0]/a, b_pos, b_height, b_sol, b_width, b_slope))
Qe = CellVariable(name="Qe", mesh=mesh, value=np.exp(-((mesh.cellCenters.value/a-H0)/(Hw))**2)[0])
Qe = Qe * Qe_tot/((mesh.cellVolumes*Qe.value).sum() * V)

Gi = np.cumsum(mesh.cellVolumes*Qe.value) / (mesh.cellCenters.value[0] + mesh.dx/2)
Gi = np.append([Gi[0]/2],(Gi[1:]+Gi[:-1])/2)

print('Volume = %s m^3' % (mesh.cellVolumes.sum() * V))
print('Heating power = %0.3e W' % ((mesh.cellVolumes*Qe).sum() * V))

Te.constrain(Te_bc, mesh.facesRight)

Now set up the values for the stochastization model.

Note that we are implementing the randomize model on the log of the fluxes!

In [ ]:
s2 = 0.2; s3 = 0.2; d2 = 10; d3 = 10
N1 = 10; N2 = 90; N3 = 30
l1 = np.log10(Gi[Gi.shape[0]//2])
l2 = l1; l3 = l1
x1 = l1; x2 = l1**2

alpha = 0.001
NITER = 1000
T = dt*(np.arange(NITER)+1)

Run the model for NITER steps.

- the initial $\chi$ is the default (1.0)
- one step is performed
- the updated temperature profile is used to recalculate $\chi$
- another step is performed, $\chi$ is recalculated, and this is then repeated

$$
\chi_{new} = \frac{\Gamma_{new}}{\Gamma_{target}} * \left(\beta + (1-\beta) \left(\frac{\nabla T_{new}}{\nabla T_{target}}\right)^\gamma\right) * \chi
$$

where, for example, $\beta=0.1$ and $\gamma=4$

In [ ]:
if plots: 
    viewer = Viewer(vars=(Te), title='Heating power = %0.3e W\nchi = %s' % (Qe.cellVolumeAverage.value * V, chi), 
                    datamin=0, datamax=5000)

chi_run = chi
l1_c = x1/10
x1_c = x1/10
beta = 0.1
gamma = 0.0

Te_history = []
runs = {}
for N in np.arange(0,NITER):
    eqI = TransientTerm(coeff=scipy.constants.e*ne*1.5) == DiffusionTerm(coeff=scipy.constants.e*ne*chi_run) + Qe
    eqI.solve(var=Te, dt=dt)
    if plots: viewer.plot()
    Te_history.append(Te.value.copy())
    Gd = -np.gradient(Te.value, mesh.cellCenters.value[0]) * ne.value * eV
    runs[N] = {}
    runs[N]['x'], runs[N]['mean'], runs[N]['std'], runs[N]['xrange'], runs[N]['x1'], runs[N]['x2'] = ComPat.randomize(N1, N2, N3, l1=l1_c, l2=l2, l3=l3, s2=s2, s3=s3, d2=d2, d3=d3, x1=x1_c, x2=x2, alpha=alpha)
    runs[N]['gTnorm'] = Gd[Gd.shape[0]//2] / Gd_ss[Gd_ss.shape[0]//2]
    x1_c = runs[N]['x1']
    l1_c = x1_c
#    runs[N]['chi'] = 10 ** runs[N]['x1'] / Gi[Gi.shape[0]//2]
    runs[N]['chi'] = (10 ** runs[N]['x1'] / Gi[Gi.shape[0]//2]) * (beta + (1.0 - beta) * runs[N]['gTnorm']**gamma) * chi
    chi_run = runs[N]['chi']

## All Te profiles

In [ ]:
plt.figure()
Te_all =  np.array(Te_history)
plt.plot(mesh.cellCenters.value[0], Te_all.T)
plt.xlabel('rho')
plt.ylabel('Te')
plt.title('dt = %s, alpha = %s, NITER = %s' % (dt, alpha, NITER))
plt.savefig('Te_snapshots_dt=%s_alpha=%s.png' % (dt, alpha))

## Central Te versus iteration number

In [ ]:
plt.figure()
plt.plot(T, Te_all[:,0])
plt.xlabel('Time [s]')
plt.ylabel('Central Te [eV]')
plt.title('dt = %s, alpha = %s' % (dt, alpha))
plt.savefig('Te_0_dt=%s_alpha=%s.png' % (dt, alpha))

In [ ]:
plt.figure()
plt.plot(T, [r['chi'] for r in runs.values()])
#print([r['chi'] for r in runs.values()])
plt.xlabel('Time [s]')
plt.ylabel('chi [$m^2 s^{-1}$]')
plt.title('dt = %s, alpha = %s' % (dt, alpha))
plt.savefig('chi_dt=%s_alpha=%s.png' % (dt, alpha))

## Flux versus the iteration number, together with steady state "target" flux (determined by integration of the source term).

In [ ]:
plt.figure()
plt.plot(T, 10**np.array([r['x1'] for r in runs.values()]))
plt.xlabel('macro iteration')
plt.ylabel('flux')
plt.plot([T.min(),T.max()], [Gi[Gi.shape[0]//2], Gi[Gi.shape[0]//2]])
plt.xlabel('Time [s]')
plt.ylabel('flux')
plt.title('dt = %s, alpha = %s' % (dt, alpha))
plt.savefig('Flux_dt=%s_alpha=%s.png' % (dt, alpha))
print('All iterations: flux mean = %s, std-dev = %s, ref = %s' % 
      ((10**np.array([r['x1'] for r in runs.values()])).mean(), 
       (10**np.array([r['x1'] for r in runs.values()])).std(),
       Gi[Gi.shape[0]//2]))

## Histogram of the central Te over the complete set of iterations

In [ ]:
plt.figure()
plt.hist(Te_all[:,0], bins=20)
plt.xlabel('Te[0]')
plt.ylabel('count')
plt.title('Histogram of Te_0 over all iterations\ndt = %s, alpha = %s' % (dt, alpha))
plt.savefig('Te_0_histogram_all_dt=%s_alpha=%s.png' % (dt, alpha))
print('All iterations: Te_0 mean = %s, std-dev = %s' % (np.mean(Te_all, axis=0)[0], np.std(Te_all, axis=0)[0]))

## Histogram of the central Te over the last 50% of the iterations

In [ ]:
plt.figure()
plt.hist(Te_all[NITER//2:,0], bins=20)
plt.xlabel('Te[0]')
plt.ylabel('count')
plt.title('Histogram of Te_0 over last 50%% iterations\ndt = %s, alpha = %s' % (dt, alpha))
plt.savefig('Te_0_histogram_last_50%%_dt=%s_alpha=%s.png' % (dt, alpha))
print('Last 50%% iterations: Te_0 mean = %s, std-dev = %s' % (np.mean(Te_all[NITER//2:], axis=0)[0], np.std(Te_all[NITER//2:], axis=0)[0]))

## Plot of mean and standard deviation of Te over all iterations

In [ ]:
plt.figure()
plt.plot(mesh.cellCenters.value[0], Te_all.mean(axis=0), 'b', label='All iterations')
plt.fill_between(mesh.cellCenters.value[0], 
                 Te_all.mean(axis=0)-Te_all.std(axis=0), 
                 Te_all.mean(axis=0)+Te_all.std(axis=0), color='blue', alpha=0.5)
plt.xlabel('rho')
plt.ylabel('Te')
plt.legend(loc=0)
plt.title('dt = %s, alpha = %s, NITER = %s' % (dt, alpha, NITER))
plt.savefig('Te_mean_stddev_all_dt=%s_alpha=%s.png' % (dt, alpha))

## Plot of mean and standard deviation of Te over the last 50% of the iterations

In [ ]:
plt.figure()
plt.plot(mesh.cellCenters.value[0], Te_all[NITER//2:].mean(axis=0), 'r', label='Last 50% iterations')
plt.fill_between(mesh.cellCenters.value[0], 
                 Te_all[NITER//2:].mean(axis=0)-Te_all[NITER//2:].std(axis=0), 
                 Te_all[NITER//2:].mean(axis=0)+Te_all[NITER//2:].std(axis=0), color='red', alpha=0.5)
plt.xlabel('rho')
plt.ylabel('Te')
plt.legend(loc=0)
plt.title('dt = %s, alpha = %s, NITER = %s' % (dt, alpha, NITER))
plt.savefig('Te_mean_stddev_last_50%%_dt=%s_alpha=%s.png' % (dt, alpha))

In [ ]:
with open("results_test.log", "a+") as file:
    print(N1, N2, N3, s2, s3, d2, d3, alpha, beta, gamma, dt, NITER, 
          (10**np.array([r['x1'] for r in runs.values()])).mean(), 
          (10**np.array([r['x1'] for r in runs.values()])).std(),
          Gi[Gi.shape[0]//2],
          np.mean(Te_all, axis=0)[0], np.std(Te_all, axis=0)[0],
          np.mean(Te_all[NITER//2:], axis=0)[0], np.std(Te_all[NITER//2:], axis=0)[0],
         file=file, sep=',')